### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages if want to see what's going on in detail
# logging.basicConfig(level=logging.DEBUG)
# turn off debug messages, only show info and higher level messages
logging.basicConfig(level=logging.INFO)

### Create vessel - add VesselProperties and ConsumesEnergy mixins

In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Vessel M9',
    "route": None,
    "geometry": None,
    "v": None,  # m/s
    "type": None,
    "B": 14.3,
    "L": 135,
    "H_e": None, 
    "H_f": None, 
    "T": 1.5,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": False, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "P_installed": 2236,   #assume
    "P_tot_given": None, # kW
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 2,# number of propellers
    "L_w": 3.0,
    "C_B":0.8, 
    "C_year": 2000,
}             



In [4]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)

### Set vessel properties and sailing conditions ( input value )

In [5]:
V_s = [2.78, 4.17, 6.94]         # ship sailing speeds to water, (m/s)  
h_0 = [15]                 # water depths,(m)
# C_year = [ 1980,1990, 2000, 2010]   # engine construction years


### prepare input matrix for calculation

In [6]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(h_0, V_s))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"h_0": item[0], "V_s": item[1]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.head(20)

,h_0,V_s
0,15,2.78
1,15,4.17
2,15,6.94


### Run OpenTNSim to Calculate resistance, required power and emissions 

In [7]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    # C_year = row['C_year']
    # data_vessel_i = data_vessel.copy()
    # data_vessel_i['C_year'] = C_year
    vessel = TransportResource(**data_vessel_i)
    
    V_s = row['V_s']
    h_0 = row['h_0']
    vessel.calculate_properties() # L is used here in the computation of L_R
    h_0 = vessel.calculate_h_squat(v = V_s, h_0 = h_0)
    print(h_0)
    R_f = vessel.calculate_frictional_resistance(V_s, h_0) 
    R_f_one_k1 = vessel.calculate_viscous_resistance()
    R_APP = vessel.calculate_appendage_resistance(V_s)
    R_W = vessel.calculate_wave_resistance(V_s, h_0)
    R_res = vessel.calculate_residual_resistance(V_s, h_0)
    R_T = vessel.calculate_total_resistance(V_s, h_0)
    P_propulsion = vessel.calculate_total_power_required(V_s, h_0)

    vessel.emission_factors_general()
    vessel.correction_factors(V_s, h_0)
    vessel.calculate_emission_factors_total(V_s, h_0)
    vessel.calculate_SFC_final(V_s, h_0)
    Fuel_g_m, Fuel_g_m_C_year = vessel.calculate_diesel_use_g_m(V_s)
    Fuel_g_s, Fuel_g_s_C_year = vessel.calculate_diesel_use_g_s()
    # [emission_g_m_CO2,emission_g_m_PM10,emission_g_m_NOX] = vessel.calculate_emission_rates_g_m(V_s)
    
    result = {}
    result.update(row)
    
    result['R_f'] = R_f    
    # result['R_f_one_k1'] = R_f_one_k1
    # result['R_APP'] = R_APP
    result['R_W'] = R_W
    # result['R_res'] = R_res
    result['R_T'] = R_T
    result['P_installed'] = vessel.P_installed
    result['P_propulsion'] = P_propulsion
    # result['P_propulsion'] = P_tot - vessel.P_hotel_perc * vessel.P_installed    
    # result['Fuel_g_km'] = Fuel_g_m * 1000
    result['Fuel_L_per_hour'] = Fuel_g_s_C_year * 3.6
    # result['emission_g_km_CO2'] = emission_g_m_CO2 * 1000
    # result['emission_g_km_PM10'] = emission_g_m_PM10 * 1000
    # result['emission_g_km_NOX'] = emission_g_m_NOX * 1000
    results.append(result)

3it [00:00, 43.12it/s]

15.0
0.55
0.55
0.55
0.55
15.0
0.78
0.78
0.78
0.78
15.0
0.65
0.65
0.65
0.65


In [8]:
plot_df = pd.DataFrame(results)


# convert from meters per second to km per hour
ms_to_kmh = 3.6
plot_df['V_s_km'] = plot_df['V_s'] * ms_to_kmh

plot_df.head(7)

,h_0,V_s,R_f,R_W,R_T,P_installed,P_propulsion,Fuel_L_per_hour,V_s_km
0,15.0,2.78,12.720819,9.652501,32.637620,2236,"(164.96833516105457, 164.96833516105457, 164.9...",41.629818,10.008
1,15.0,4.17,27.098764,57.564928,106.162361,2236,"(567.5603139056136, 567.5603139056136, 567.560...",131.297173,15.012
2,15.0,6.94,70.246470,250.730094,372.724621,2236,"(3979.5521042202113, 3979.5521042202113, 2236)",467.547600,24.984
